In [2]:
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception_type
from langchain.llms import OpenAI
from langchain import HuggingFaceHub,LLMChain
from langchain.prompts import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import pandas as pd
from collections import Counter
from io import StringIO
import streamlit as st
import re
import os
import openai
import ast
import json
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
# Decorator for automatic retry requests
@retry(
    retry = retry_if_exception_type((openai.APIError, openai.APIConnectionError, openai.RateLimitError, openai.Timeout)),
    # Function to add random exponential backoff to a request
    wait = wait_random_exponential(multiplier = 1, max = 60),
    stop = stop_after_attempt(5)
)
def run_llm_chain(hub_chain,genre, parameters,text):
    output =hub_chain.run(genre=genre,parameters=parameters,text=text)
    print(output)     
    return output

In [3]:
from pydantic import BaseModel, Field, conlist
from typing import List, Optional, Tuple
class OutputResult(BaseModel):
    score: int = Field(description="The score for the text")
    explanation:str = Field(description="The explanation for the score")

In [5]:
from langchain.llms import OpenAI
from langchain import HuggingFaceHub,LLMChain
from langchain.prompts import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import pandas as pd
from collections import Counter
from io import StringIO
import streamlit as st
import re
import os
import openai
import ast
import json
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator


def evaluateDataset(genre,dataframe) -> pd.DataFrame:
    import os
    import openai
    import ast
    from langchain.chat_models import ChatOpenAI
    # initialize the models
    openai.api_key = os.getenv("OPENAI_API_KEY")
    openai = ChatOpenAI(             
        model_name="gpt-3.5-turbo",
        openai_api_key=openai.api_key,
        temperature=1.5
    )  

    
    examples = [          
            {
                "genre":"sci-fi story",
                "parameters": "Alpha Galaxy Explorer, Celestial Iris, pursuit of ancient artifacts in a parallel universe",
                "text": "In the vast depths of the Alpha Galaxy, nestled within the glimmering expanse of Celestial Iris, resides our remarkable protagonist, the Alpha Galaxy Explorer. With a boundless curiosity and unwavering determination, our explorer embarks on an extraordinary adventure in search of ancient artifacts in a parallel universe. Journeying through shimmering stardust and cosmic membranes, they unlock the secrets of a forgotten civilization, facing ethereal creatures and unraveling the celestial architecture. Each artifact brings them closer to uncovering the truth of their own parallel existence, teetering between surreal polarities of light and darkness, and transcending the barriers of reality itself.",
                "output": OutputResult.model_validate({
                    "score": 95, 
                    "explanation":"The text is fluent and coherent, it is perfecty aligned and based on the input text parameters and matches the genre. It is interesting and creative. It is a bit too general and doesn't provide a lot of detail on the adventure itself thats why I reduce the score from 100. Overall very good quality"                    
                     }).model_dump_json().replace("{", "{{").replace("}", "}}"),
            },
             {
                "genre":"fairytale",
                "parameters": "Gandalf the Great, Magiglot Woodlands, Quest of the Whispering Moonstone",
                "text" : "Gandalf the Great, Magiglot Woodlands, Quest of the Whispering Moonstone, and the Quest of the Golden Apple. Gandalf the Great Gandalf the Great is a quest in The Elder Scrolls V: Skyrim. Background class=sprite edit-pencil />Edit Gandalf the Great is a legendary wizard who is the leader of the Fellowship of the Ring. He is the only known member of the Fellowship to have survived the War of the Ring. The quest begins with the player being summoned to the Hall of Fire by the High Elf, Alduin. Alduin is the leader of the White-Gold Elves, and is the only known member of the White-Gold Elves to have survived the War of the Ring. The player is then given the task of retrieving the Golden Apple from the Hall of Fire. The Golden Apple is a powerful magical item that can be used to summon Alduin. The player is then given the task of retrieving the Whispering Moonstone from the Hall of Fire. The Whispering Moonstone is a powerful magical item that can be used to summon Alduin.",
                "output": OutputResult.model_validate({
                    "score":10, 
                    "explanation":"The text is inconsistent, not fluent and not very coherent.I gave it a score of 10 because it does mention the input parameters - the name of the magical character 'Gandalf the Great' and the magical artifacts names"
                     }).model_dump_json().replace("{", "{{").replace("}", "}}"),
            },
            {
                "genre":"fairytale",
                "parameters":"Santriel the Beautiful, Magic Forest, Quest for Moonstone",
                "text" :"Santriel the Beautiful, Magic Forest, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone, Quest for Moonstone->Moonstone",
                "output": OutputResult.model_validate({
                    "score":0, 
                    "explanation":"The provided text is not fluent and there is no coherent development of a storyline."
                     }).model_dump_json().replace("{", "{{").replace("}", "}}"),
            },   
            {
                "genre":"fairytale",
                "parameters":"Santriel the Beautiful, Magic Forest, Quest for Moonstone",
                "text" :"In the enchanting Magic Forest, there lived a majestic and mysterious character named Seraphina, known for her captivating beauty and unparalleled magical powers. One day, Seraphina embarked on a perilous journey to find the elusive Moonstone, said to possess the power to grant wishes. As she ventured deeper into the forest, she encountered various challenges and obstacles, including treacherous paths, mischievous creatures, and ancient spells. Despite these challenges, Seraphina remained steadfast in her determination, using her cunning and magical abilities to overcome each hurdle. With her unwavering courage and unshakeable belief in her mission, she finally reached the Moonstone, which glowed with an ethereal light. As she touched the Moonstone, Seraphina felt an overwhelming sense of power and wisdom, and she knew that her journey had been worth it. With the Moonstone in hand, she returned to the Magic Forest, where she was hailed as a hero and celebrated for her bravery and determination. From that day on, Seraphina's legend grew, inspiring countless others to embark on their own quests for the Moonstone, and the Magic Forest became a symbol of hope and magic, where anything was possible.",
                "output": OutputResult.model_validate({
                    "score":70, 
                    "explanation":"The provided text is well written, consistent and fluent. I reduced the score from 100 because it doesn't fully follow the input parameters - the requested name of the magical character in the input parameters is 'Santriel the Beautiful' as the name of the character in the story is 'Seraphina'"
                     }).model_dump_json().replace("{", "{{").replace("}", "}}"),
            }, 
            {
                "genre":"news item",
                "parameters":"GizmoDo, The Buddy Team, lyrical",
                "text" :"GizmoDo, Oh oh oh! The Buddy Team, La lala lala! It's the newest gadget that you won't want to miss, With features that'll make you say, 'What is this?!' Catchy and cool, it'll make you groove, GizmoDo, the perfect tech move! Order one today and be part of the fun, Mac or PC, it works with everyone!",
                "output": OutputResult.model_validate({
                    "score":0, 
                    "explanation":"The provided text does not match the requested genre."
                     }).model_dump_json().replace("{", "{{").replace("}", "}}"),
            }, 
            {
                "genre":"superhero comics",
                "parameters":"superhero name, location, villain",
                "text" :"The [superhero name] embarked on a thrilling adventure in the his native [location]. With [superhero name]'s powers, he overcame every challenge that came his way and ultimately defeated the [villain]. The residents of [location] forever praised [superhero name]'s bravery and kindness, making them a legendary hero honored for generations to come.",
                "output": OutputResult.model_validate({
                    "score":0, 
                    "explanation":"The provided text is a template without content, it is too generic and lacks specificity and interest."
                     }).model_dump_json().replace("{", "{{").replace("}", "}}"),
            }, 
        ]


    

    # create a example template
    example_template = """
        User: 
        genre: {genre},
        parameters: {parameters},
        text:{text}.
        AI: {output}
    """
    # create a prompt example from above template
    example_prompt = PromptTemplate(
        input_variables=["genre","parameters","text","output"],
        template=example_template
    )

    parser = PydanticOutputParser(pydantic_object=OutputResult)

    # now break our previous prompt into a prefix and suffix
    # the prefix is our instructions    
    prefix = """You are a teacher grading a story text written by a student. Given a genre for the text, set of parameters on which the text should be based, and a text written by the student based on those parameters in the specified genre, you need to score the text on a continual scale from 0 (worst) to 100 (best),
                where a score of 0 means 'The given text makes no sense and is totally not understandable or is not related to the given text parameters or is not related to the specified genre' and a score of 100 means 
            'The given text matches the specified genre, is perfect-written, fluent and coherent and highly consistent with the given text parameters'. Deduce ALL points if the text doesn't match the input parameters. Deduce ALL points if either the input parameters or the text are generic,template-like or lacks specifity and details.
            You also need to provide a short one-sentence explanation for your score.
            The result should be a json format with two attributes, score and explanation. Transform the output into structured object given those instructions: {format_instructions}. Please use the following examples as a reference for your evaluation.:
    """

    # and the suffix our user input and output indicator
    suffix = """
    User: 
        genre: {genre},
        parameters: {parameters},
        text:{text}.
    AI:"""


    # now create the few shot prompt template
    few_shot_prompt_template = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix=prefix,
        suffix=suffix,
        input_variables=["genre","parameters","text"],
        example_separator="\n\n",
        partial_variables={"format_instructions": parser.get_format_instructions()},    
    )

    hub_chain = LLMChain(prompt=few_shot_prompt_template,llm=openai,verbose=True) 
    df = pd.DataFrame()
    for index, row in dataframe.iterrows():
            parameters_value = row['parameters']
            text_value = row['text']
                         
            output  = run_llm_chain(hub_chain,genre, parameters_value,text_value)                          
            # # Extract the first and second elements as strings
            try:
                parsed_result = parser.parse(output)
                
            except Exception as e:
                print(e)
                continue
           
            first_string = parsed_result.score if parsed_result.score else 'Not specified'
            print('score:',first_string)

            second_string = parsed_result.explanation if parsed_result.explanation else 'Not specified'
            print('explanation:',second_string)      
            
            

            # Access and print the key-value pairs
            
            new_row = {
            'genre': genre,
            'parameters':parameters_value, 
            'text':text_value, 
            'score':first_string,
            'explanation':second_string
            }
            new_row = pd.DataFrame([new_row])
            df = pd.concat([df, new_row], axis=0, ignore_index=True)
    
    return df

### Evaluation of different prompts
#### Simple prompt -no role, no style suggestions, no examples

In [7]:
csv_file_path = '../../datasets/evaluation/simple_fairytale.csv'
df = pd.read_csv(csv_file_path)
another_df=evaluateDataset("fairytale",df)

c:\Users\sbobu\Desktop\computer project\storygen\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\sbobu\Desktop\computer project\storygen\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\sbobu\Desktop\computer project\storygen\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  wa



> Entering new LLMChain chain...
Prompt after formatting:
You are a teacher grading a story text written by a student. Given a genre for the text, set of parameters on which the text should be based, and a text written by the student based on those parameters in the specified genre, you need to score the text on a continual scale from 0 (worst) to 100 (best),
                where a score of 0 means 'The given text makes no sense and is totally not understandable or is not related to the given text parameters or is not related to the specified genre' and a score of 100 means 
            'The given text matches the specified genre, is perfect-written, fluent and coherent and highly consistent with the given text parameters'. Deduce ALL points if the text doesn't match the input parameters. Deduce ALL points if either the input parameters or the text are generic,template-like or lacks specifity and details.
            You also need to provide a short one-sentence explanation for your

In [8]:
another_df

,genre,parameters,text,score,explanation
0,fairytale,"fairy, enchanted forest, rescue mission","Once upon a time in the enchanted forest, twin...",95,"The text is well-written, coherent, and engagi..."
1,fairytale,"magical character, location, adventure","Once upon a time, a magical character named {{...",Not specified,The text provided is a generic and template-li...
2,fairytale,"Brave Unicorn, Enchanted Forest, Quest","In the heart of the Enchanted Forest, the Brav...",90,"The text is well-written, consistent with the ..."
3,fairytale,"fairytale, enchanted forest, rescue mission","Once upon a time, in an enchanted forest, ther...",100,"The text is fluently written, coherent, and pe..."
4,fairytale,"elf, enchanted forest, saves the kingdom","In the heart of the enchanted forest, an elf n...",100,"The text is fluent and coherent, perfectly ali..."
5,fairytale,"fairy, enchanted forest, rescuing a lost unicorn","Once upon a time, in the heart of the enchante...",90,The text is well-written and consistent with t...
6,fairytale,"magical character, location, adventure","Once upon a time, the magical character embark...",Not specified,The given text is extremely generic and lacks ...
7,fairytale,"Poppy the Unicorn, Enchanted Forest, Saving th...",In the depths of the Enchanted Forest lived Po...,95,"The text is fluent and coherent, it aligns per..."
8,fairytale,"magical character, location, adventure","Once upon a time, the mystical [magical charac...",Not specified,The provided text does not include the magical...
9,fairytale,"fairy, enchanted forest, ancient tree, dragon ...","In the depths of the enchanted forest, the fai...",85,"The text is well-written and coherent, maintai..."


In [9]:
another_df['score'] = another_df['score'].replace(to_replace=['Not specified', 'not specified'], value=0, regex=True)
another_df

C:\Users\sbobu\AppData\Local\Temp\ipykernel_107608\3509848069.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  another_df['score'] = another_df['score'].replace(to_replace=['Not specified', 'not specified'], value=0, regex=True)


,genre,parameters,text,score,explanation
0,fairytale,"fairy, enchanted forest, rescue mission","Once upon a time in the enchanted forest, twin...",95,"The text is well-written, coherent, and engagi..."
1,fairytale,"magical character, location, adventure","Once upon a time, a magical character named {{...",0,The text provided is a generic and template-li...
2,fairytale,"Brave Unicorn, Enchanted Forest, Quest","In the heart of the Enchanted Forest, the Brav...",90,"The text is well-written, consistent with the ..."
3,fairytale,"fairytale, enchanted forest, rescue mission","Once upon a time, in an enchanted forest, ther...",100,"The text is fluently written, coherent, and pe..."
4,fairytale,"elf, enchanted forest, saves the kingdom","In the heart of the enchanted forest, an elf n...",100,"The text is fluent and coherent, perfectly ali..."
5,fairytale,"fairy, enchanted forest, rescuing a lost unicorn","Once upon a time, in the heart of the enchante...",90,The text is well-written and consistent with t...
6,fairytale,"magical character, location, adventure","Once upon a time, the magical character embark...",0,The given text is extremely generic and lacks ...
7,fairytale,"Poppy the Unicorn, Enchanted Forest, Saving th...",In the depths of the Enchanted Forest lived Po...,95,"The text is fluent and coherent, it aligns per..."
8,fairytale,"magical character, location, adventure","Once upon a time, the mystical [magical charac...",0,The provided text does not include the magical...
9,fairytale,"fairy, enchanted forest, ancient tree, dragon ...","In the depths of the enchanted forest, the fai...",85,"The text is well-written and coherent, maintai..."


In [10]:
#write to file and calculate average score
csv_file_path = '../..//datasets/evaluation/simple_fairytale_evaluations.csv'

# Save the selected columns with custom column names to a CSV file
another_df.to_csv(csv_file_path, index=False)
another_df['score'] = pd.to_numeric(another_df['score'], errors='coerce')

# Calculate the average score across all numeric values in the 'score' column
average_score = another_df['score'].mean()
print("average score for simple df: ",average_score)

average score for simple df:  79.25531914893617


#### Elaborate prompt evaluation - with role, style

In [11]:
csv_file_path = '../../datasets/evaluation/elaborate_fairytale.csv'
df = pd.read_csv(csv_file_path)
another_df=evaluateDataset("fairytale",df)



> Entering new LLMChain chain...
Prompt after formatting:
You are a teacher grading a story text written by a student. Given a genre for the text, set of parameters on which the text should be based, and a text written by the student based on those parameters in the specified genre, you need to score the text on a continual scale from 0 (worst) to 100 (best),
                where a score of 0 means 'The given text makes no sense and is totally not understandable or is not related to the given text parameters or is not related to the specified genre' and a score of 100 means 
            'The given text matches the specified genre, is perfect-written, fluent and coherent and highly consistent with the given text parameters'. Deduce ALL points if the text doesn't match the input parameters. Deduce ALL points if either the input parameters or the text are generic,template-like or lacks specifity and details.
            You also need to provide a short one-sentence explanation for your

In [12]:
another_df

,genre,parameters,text,score,explanation
0,fairytale,"unicorn, enchanted forest, rescue mission, spe...","Once upon a time, in the mystical enchanted fo...",90,"The text is fluently written and coherent, ali..."
1,fairytale,"Fairy princess, Enchanted forest, Seeking the ...","In the heart of the Enchanted forest, the Fair...",80,"The text is well-written, coherent, and consis..."
2,fairytale,"Pixie, Enchanted Forest, Mystical Quest","In the heart of the Enchanted Forest, a mischi...",80,The text is well written and aligned with the ...
3,fairytale,"Humble elf, Enchanted forest, Epic journey",Once upon a time in the mysterious realm of th...,85,The text is well-written and consistent with t...
4,fairytale,"dragons, enchanted forest, treasure hunt",In a magical land where dragons soared through...,90,"The text is well-written, engaging, and cohere..."
5,fairytale,"mermaid, enchanted forest, hide and seek with ...","Once upon a time in an enchanted forest, a pla...",90,The text is well-written and immerses the read...
6,fairytale,"Enchanting Wizard, Crystal Cavern, Epic Expedi...","Once upon a time, the Enchanting Wizard embark...",100,The text is perfectly aligned with the specifi...
7,fairytale,"elf, enchanted forest, ancient curse, jewelry","Once upon a time in the enchanted forest, the ...",80,"The text is well-written and coherent, followi..."
8,fairytale,"enchanting unicorn, enchanted forest, quest","In the depths of the enchanted forest, the enc...",90,"The text is well-written, fluent, and consiste..."
9,fairytale,"magical character, location, adventure","Once upon a time, in a mystical land called [l...",80,While the text follows the fairytale genre and...


In [13]:
another_df['score'] = another_df['score'].replace(to_replace=['Not specified', 'not specified'], value=0, regex=True)
another_df

C:\Users\sbobu\AppData\Local\Temp\ipykernel_107608\3509848069.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  another_df['score'] = another_df['score'].replace(to_replace=['Not specified', 'not specified'], value=0, regex=True)


,genre,parameters,text,score,explanation
0,fairytale,"unicorn, enchanted forest, rescue mission, spe...","Once upon a time, in the mystical enchanted fo...",90,"The text is fluently written and coherent, ali..."
1,fairytale,"Fairy princess, Enchanted forest, Seeking the ...","In the heart of the Enchanted forest, the Fair...",80,"The text is well-written, coherent, and consis..."
2,fairytale,"Pixie, Enchanted Forest, Mystical Quest","In the heart of the Enchanted Forest, a mischi...",80,The text is well written and aligned with the ...
3,fairytale,"Humble elf, Enchanted forest, Epic journey",Once upon a time in the mysterious realm of th...,85,The text is well-written and consistent with t...
4,fairytale,"dragons, enchanted forest, treasure hunt",In a magical land where dragons soared through...,90,"The text is well-written, engaging, and cohere..."
5,fairytale,"mermaid, enchanted forest, hide and seek with ...","Once upon a time in an enchanted forest, a pla...",90,The text is well-written and immerses the read...
6,fairytale,"Enchanting Wizard, Crystal Cavern, Epic Expedi...","Once upon a time, the Enchanting Wizard embark...",100,The text is perfectly aligned with the specifi...
7,fairytale,"elf, enchanted forest, ancient curse, jewelry","Once upon a time in the enchanted forest, the ...",80,"The text is well-written and coherent, followi..."
8,fairytale,"enchanting unicorn, enchanted forest, quest","In the depths of the enchanted forest, the enc...",90,"The text is well-written, fluent, and consiste..."
9,fairytale,"magical character, location, adventure","Once upon a time, in a mystical land called [l...",80,While the text follows the fairytale genre and...


In [15]:
#write to file and calculate average score
csv_file_path = '../../datasets/evaluation/elaborate_fairytale_evaluations.csv'

# Save the selected columns with custom column names to a CSV file
another_df.to_csv(csv_file_path, index=False)
another_df['score'] = pd.to_numeric(another_df['score'], errors='coerce')

# Calculate the average score across all numeric values in the 'score' column
average_score = another_df['score'].mean()
print("average score for simple df: ",average_score)

average score for simple df:  87.44680851063829


#### Evaluation of prompt with few-shot examples

In [16]:
csv_file_path = '../../datasets/evaluation/examples_fairytale.csv'
df = pd.read_csv(csv_file_path)
another_df=evaluateDataset("fairytale",df)



> Entering new LLMChain chain...
Prompt after formatting:
You are a teacher grading a story text written by a student. Given a genre for the text, set of parameters on which the text should be based, and a text written by the student based on those parameters in the specified genre, you need to score the text on a continual scale from 0 (worst) to 100 (best),
                where a score of 0 means 'The given text makes no sense and is totally not understandable or is not related to the given text parameters or is not related to the specified genre' and a score of 100 means 
            'The given text matches the specified genre, is perfect-written, fluent and coherent and highly consistent with the given text parameters'. Deduce ALL points if the text doesn't match the input parameters. Deduce ALL points if either the input parameters or the text are generic,template-like or lacks specifity and details.
            You also need to provide a short one-sentence explanation for your

In [17]:
another_df

,genre,parameters,text,score,explanation
0,fairytale,"Rainbow Unicorn, Crystal Forest, protecting th...","In the mystical land of Enchanted Realm, a bea...",100,The text is perfectly aligned with the specifi...
1,fairytale,"Moonbeam, enchanted forest, saving the lost un...","Moonbeam, a unicorn with a gleaming silver coa...",85,"The text is well-written and detailed, providi..."
2,fairytale,"Sapphire the Mermaid, Crystal Cove, trapping a...",In the magical waters of Crystal Cove lived Sa...,80,"The text is well-written and coherent, with a ..."
3,fairytale,"Pip the Enchanted Bread Loaf, Sugar Rush Kingd...","In the whimsical Sugar Rush Kingdom, Pip the E...",90,The text is well-written with creativity and e...
4,fairytale,"Starlight Fairy, Crystal Caves, defeating evil...",Starlight Fairy lived among the magnificent Cr...,85,"The text is fluent, coherent, and well-written..."
5,fairytale,"Crystal Rose, Enchanted Forest, defeating the ...","In the heart of the Enchanted Forest, the magi...",85,The text effectively captures the essence of a...
6,fairytale,"Luna the Water Sprite, Crystal Caverns, findin...",In the depths of the Crystal Caverns lived Lun...,85,"The text is well-written and coherent, matchin..."
7,fairytale,"Luna the Star Weaver, Glowing Garden of Ephema...","In the magical Glowin Garden of Ephemaria, res...",95,The text is extremely well-written and engagin...
8,fairytale,"Iris the Enchanted Flower, Crystal Caverns, fi...","In the heart of the Crystal Caverns, there liv...",80,"The text is well-written and coherent, maintai..."
9,fairytale,"Saga Linelius, Amethyst Canyon, recovering an ...",Deep within the depths of the Amethyst Canyon ...,90,"The text is well written and flows nicely, ali..."


In [18]:
another_df['score'] = another_df['score'].replace(to_replace=['Not specified', 'not specified'], value=0, regex=True)
another_df

,genre,parameters,text,score,explanation
0,fairytale,"Rainbow Unicorn, Crystal Forest, protecting th...","In the mystical land of Enchanted Realm, a bea...",100,The text is perfectly aligned with the specifi...
1,fairytale,"Moonbeam, enchanted forest, saving the lost un...","Moonbeam, a unicorn with a gleaming silver coa...",85,"The text is well-written and detailed, providi..."
2,fairytale,"Sapphire the Mermaid, Crystal Cove, trapping a...",In the magical waters of Crystal Cove lived Sa...,80,"The text is well-written and coherent, with a ..."
3,fairytale,"Pip the Enchanted Bread Loaf, Sugar Rush Kingd...","In the whimsical Sugar Rush Kingdom, Pip the E...",90,The text is well-written with creativity and e...
4,fairytale,"Starlight Fairy, Crystal Caves, defeating evil...",Starlight Fairy lived among the magnificent Cr...,85,"The text is fluent, coherent, and well-written..."
5,fairytale,"Crystal Rose, Enchanted Forest, defeating the ...","In the heart of the Enchanted Forest, the magi...",85,The text effectively captures the essence of a...
6,fairytale,"Luna the Water Sprite, Crystal Caverns, findin...",In the depths of the Crystal Caverns lived Lun...,85,"The text is well-written and coherent, matchin..."
7,fairytale,"Luna the Star Weaver, Glowing Garden of Ephema...","In the magical Glowin Garden of Ephemaria, res...",95,The text is extremely well-written and engagin...
8,fairytale,"Iris the Enchanted Flower, Crystal Caverns, fi...","In the heart of the Crystal Caverns, there liv...",80,"The text is well-written and coherent, maintai..."
9,fairytale,"Saga Linelius, Amethyst Canyon, recovering an ...",Deep within the depths of the Amethyst Canyon ...,90,"The text is well written and flows nicely, ali..."


In [19]:
#write to file and calculate average score
csv_file_path = '../../datasets/evaluation/examples_fairytale_evaluations.csv'

# Save the selected columns with custom column names to a CSV file
another_df.to_csv(csv_file_path, index=False)
another_df['score'] = pd.to_numeric(another_df['score'], errors='coerce')

# Calculate the average score across all numeric values in the 'score' column
average_score = another_df['score'].mean()
print("average score for simple df: ",average_score)

average score for simple df:  87.8125
